In [80]:
import pandas as pd
import csv, sqlite3

In [81]:
df = pd.read_csv("../cleaning-data/web-cleaned-data.csv")
df.head()

,FlightNo.,Dateandtime(UTC),Launchsite,Payloadmass,Orbit,Customer,Launchoutcome,Boosterlanding,BoosterVersion,Serial
0,1,2010-06-04,CCSFS SLC-40,8933.290816,LEO,SpaceX,Success,Failure(parachute),F9 v1.0,B0003
1,2,2010-12-08,CCSFS SLC-40,8933.290816,LEO,NASA(COTS),Success,Failure(parachute),F9 v1.0,B0004
2,3,2012-05-22,CCSFS SLC-40,525.000000,LEO,NASA(COTS),Success,No attempt,F9 v1.0,B0005
3,4,2012-10-08,CCSFS SLC-40,4700.000000,LEO,NASA(CRS),Success,No attempt,F9 v1.0,B0006
4,5,2013-03-01,CCSFS SLC-40,4877.000000,LEO,NASA(CRS),Success,No attempt,F9 v1.0,B0007


In [82]:
success = ['Controlled(ocean)', 'Success(ground pad)', 'Success(drone ship)']
df['Landingstatus'] = df['Boosterlanding'].map(lambda x: 1 if x in success else 0)

In [83]:
conn = sqlite3.connect('SpaceX.db')
df.to_sql("SPACEXLAUNCH", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [84]:
%load_ext sql
conn = sqlite3.connect('SpaceX.db')
cur = conn.cursor()
%sql sqlite:///SpaceX.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1. How many launchsite does SpaceX have? How does launchsite affect landing status?

* From the first glance, most launchs was in CCSFS SLC-40, total 115 flights, second was KSC LC-39A, 57 flights, and finally was VSFB SLC-4E, 36 flights

* Success landing rate in each launch site is 75.65%, 91.22%, 86.11% respectively

* Most ocean landing was in CCSFS (5 flights) success rate 80%, VSFB also has 2 flights success rate 50%. KSC has the highest rate of success of drone ship landing (45/46 flights). VSFB and KSC also have no failure in ground pad landing (total ground pad landing was 30, and we only have 1 failiure in CCSFS)

* PayloadMass from KSC LC-39 has the average highest mass 10405.24 kg

* When observe launchsite with orbit, the most various launchsite was CCSFS SLC-40, it already tried HEO, ES-L1, BLT, MEO orbit, which the other launchsite haven't tried yet. We also have 1 sub-orbital from KSC and 1 Heliocentric from VSFB

* The special things are SSO and PO orbit almost launched from VSFB, SSO also launched from CCSFS (8 flights) but fewer when compared to VSFB

* There is no Falcon v1.0 v1.1 launched at KSC

In [85]:
%%sql
select launchsite, landingstatus, count(launchsite) as LaunchTotal
from spacexlaunch
group by launchsite, landingstatus

 * sqlite:///SpaceX.db
Done.


Launchsite,Landingstatus,LaunchTotal
CCSFS SLC-40,0,28
CCSFS SLC-40,1,87
KSC LC-39A,0,5
KSC LC-39A,1,52
VSFB SLC-4E,0,5
VSFB SLC-4E,1,31


In [86]:
%%sql
select launchsite, avg(payloadmass) as AverageMass, min(payloadmass) as MinMass, max(payloadmass) as MaxMass
from spacexlaunch
group by launchsite

 * sqlite:///SpaceX.db
Done.


Launchsite,AverageMass,MinMass,MaxMass
CCSFS SLC-40,8161.263708961844,362.0,17400.0
KSC LC-39A,10405.23827425707,325.0,16700.0
VSFB SLC-4E,9068.905045351474,400.0,16400.0


In [87]:
%%sql
select orbit, count(orbit) as TotalFlight
from spacexlaunch
where launchsite = 'CCSFS SLC-40'
group by orbit

 * sqlite:///SpaceX.db
Done.


Orbit,TotalFlight
BLT,2
ES-L1,1
GTO,34
HEO,1
LEO,62
MEO,6
PO,1
SSO,8


In [88]:
%%sql
select orbit, count(orbit) as TotalFlight
from spacexlaunch
where launchsite = 'KSC LC-39A'
group by orbit

 * sqlite:///SpaceX.db
Done.


Orbit,TotalFlight
GTO,9
LEO,46
PO,1
Sub-orbital,1


In [89]:
%%sql
select orbit, count(orbit) as TotalFlight
from spacexlaunch
where launchsite = 'VSFB SLC-4E'
group by orbit

 * sqlite:///SpaceX.db
Done.


Orbit,TotalFlight
Heliocentric,1
LEO,16
PO,8
SSO,11


In [90]:
%%sql
select boosterlanding, count(boosterlanding) as TotalFlight
from spacexlaunch
where launchsite = 'CCSFS SLC-40'
group by boosterlanding

 * sqlite:///SpaceX.db
Done.


Boosterlanding,TotalFlight
Controlled(ocean),4
Failure(drone ship),6
Failure(ground pad),1
Failure(parachute),2
No attempt,17
Precluded(drone ship),1
Success(drone ship),69
Success(ground pad),14
Uncontrolled(ocean),1


In [91]:
%%sql
select boosterlanding, count(boosterlanding) as TotalFlight
from spacexlaunch
where launchsite = 'KSC LC-39A'
group by boosterlanding

 * sqlite:///SpaceX.db
Done.


Boosterlanding,TotalFlight
Failure(drone ship),1
No attempt,4
Success(drone ship),45
Success(ground pad),7


In [92]:
%%sql
select boosterlanding, count(boosterlanding) as TotalFlight
from spacexlaunch
where launchsite = 'VSFB SLC-4E'
group by boosterlanding

 * sqlite:///SpaceX.db
Done.


Boosterlanding,TotalFlight
Controlled(ocean),1
Failure(drone ship),1
No attempt,3
Success(drone ship),22
Success(ground pad),8
Uncontrolled(ocean),1


In [93]:
%%sql
select launchsite, boosterversion, count(boosterversion) as TotalFlight
from spacexlaunch
group by launchsite, boosterversion

 * sqlite:///SpaceX.db
Done.


Launchsite,BoosterVersion,TotalFlight
CCSFS SLC-40,F9 B4,6
CCSFS SLC-40,F9 B5,81
CCSFS SLC-40,F9 FT,10
CCSFS SLC-40,F9 v1.0,5
CCSFS SLC-40,F9 v1.1,13
KSC LC-39A,F9 B4,3
KSC LC-39A,F9 B5,45
KSC LC-39A,F9 FT,9
VSFB SLC-4E,F9 B4,3
VSFB SLC-4E,F9 B5,26


2. The key customers of SpaceX

It can be seen that most of SpaceX customers are NASA, SpaceX, SES and Iridium Communications. 

* SES is a Luxembourgish leading satellite telecommunications network provider with over 70 satellites in two different orbits. 

* Iridium operates the Iridium satellite constellation, a system of 66 active satellites and nine in-orbit spares used for worldwide voice and data communication from handheld satellite phones, satellite messenger communication devices and integrated transceivers

* NASA: The National Aeronautics and Space Administration is an independent agency of the U.S. federal government responsible for the civil space program, aeronautics research, and space research. CRS are a contract solution to deliver cargo and supplies to the International Space Station (ISS) on a commmercial basis

* We have total 46 flights which are related to NASA, in which NASA(CRS) accounted for the most 26 flights but fewer total payload 71326 kgs, compared to NASA(CTS) 6 flights but total payload is 77050 kgs, average payload mass of each flights of NASA(CRS) is 2743, most of NASA launch was LEO orbit with 70% success rate (28/40)

* The average payload mass of SpaceX is approximately more than 15000 kgs each flights, total flights which is related to SpaceX are 77 flights. SpaceX focus on LEO and SSO orbit, success rate 94%, 80% respectively

* Iridium focus on PO orbit launch (7 flights) with average payload mass is 9600 kgs, it also has 1 flight for LEO orbit

* SES focus on GTO orbit launch (7 flights) with average payload mass is about 4600 kgs, it also has 1 flight for MEO orbit

In [94]:
%%sql
select customer, count(customer) as TotalFlight
from spacexlaunch
group by customer
order by TotalFlight desc
limit 5

 * sqlite:///SpaceX.db
Done.


Customer,TotalFlight
SpaceX,68
NASA(CRS),26
SES,7
Iridium Communications,7
Various,6


In [95]:
%%sql 
select customer, sum(payloadmass) as TotalPayloadMass, count(customer) as LaunchTotal, avg(payloadmass) as AveragePerLaunch
from spacexlaunch
where customer like '%NASA%'
group by customer
order by TotalPayloadMass desc

 * sqlite:///SpaceX.db
Done.


Customer,TotalPayloadMass,LaunchTotal,AveragePerLaunch
NASA(CTS),77050.0,6,12841.666666666666
NASA(CRS),71326.0,26,2743.3076923076924
NASA(CCDev),12530.0,1,12530.0
NASA(CCP),12500.0,1,12500.0
NASA(CCD),12055.0,1,12055.0
NASA(COTS),9458.290816326531,2,4729.1454081632655
Iridium Communications/GFZ/NASA,6460.0,1,6460.0
NASA/CNES,2200.0,1,2200.0
NASA(LSP),1311.0,3,437.0
NASA/NOAA/ESA/EUMETSAT,1192.0,1,1192.0


In [96]:
%%sql 
select orbit, landingstatus, count(orbit) as TotalFlights
from spacexlaunch
where customer like '%NASA%'
group by orbit, landingstatus
order by TotalFlights desc

 * sqlite:///SpaceX.db
Done.


Orbit,Landingstatus,TotalFlights
LEO,1,28
LEO,0,12
BLT,1,1
ES-L1,1,1
HEO,1,1
Heliocentric,1,1
PO,0,1
Sub-orbital,0,1


In [97]:
%%sql 
select customer, sum(payloadmass) as TotalPayloadMass, count(customer) as LaunchTotal, avg(payloadmass) as AveragePerLaunch
from spacexlaunch
where customer like '%SpaceX%'
group by customer
order by TotalPayloadMass desc

 * sqlite:///SpaceX.db
Done.


Customer,TotalPayloadMass,LaunchTotal,AveragePerLaunch
SpaceX,1056458.2908163266,68,15536.151335534216
SpaceX/Spaceflight,45432.0,3,15144.0
SpaceX/Planet Labs,30850.0,2,15425.0
SpaceX/D-Orbit,15200.0,1,15200.0
SpaceX/Capella Space/Tyvak,14000.0,1,14000.0
SpaceX/AST SpaceMobile,11938.0,1,11938.0
Hisdesat/exactEarth/SpaceX,2150.0,1,2150.0


In [98]:
%%sql 
select orbit, landingstatus, count(orbit) as TotalFlights
from spacexlaunch
where customer like '%SpaceX%'
group by orbit, landingstatus
order by TotalFlights desc

 * sqlite:///SpaceX.db
Done.


Orbit,Landingstatus,TotalFlights
LEO,1,68
LEO,0,4
SSO,1,4
SSO,0,1


In [99]:
%%sql 
select customer, orbit, sum(payloadmass) as TotalPayloadMass, count(customer) as LaunchTotal, avg(payloadmass) as AveragePerLaunch
from spacexlaunch
where customer like '%Iridium Communications%' or customer like '%SES%'
group by customer, orbit
order by TotalPayloadMass desc

 * sqlite:///SpaceX.db
Done.


Customer,Orbit,TotalPayloadMass,LaunchTotal,AveragePerLaunch
Iridium Communications,PO,57600.0,6,9600.0
SES,GTO,26855.0,6,4475.833333333333
Iridium Communications,LEO,9600.0,1,9600.0
Iridium Communications/GFZ/NASA,PO,6460.0,1,6460.0
SES/EchoStar,GTO,5400.0,1,5400.0
SES,MEO,4100.0,1,4100.0


3. How many booster does SpaceX have? Which booster has the highest average payload?

* Falcon 9 Block 5 have total 152 flights, the most active one in Falcon 9 rocket family, F9 B5 also can carry from 325 kgs to 17400 kgs payload mass which has highest average payload mass is 10625 kgs

* F9 v1.0 & F9 v1.1 have high failure rate (100% and 80% failure), for F9 B4 & F9 FT, the success rate of theses 2 rockets are better (50% and 75% success). F9 B5 highest success rate 94%

In [100]:
%%sql 
select boosterversion, 
        min(payloadmass) as MinPayloadMass,
        max(payloadmass) as MaxPayloadMass, 
        sum(payloadmass) as TotalPayloadMass, 
        avg(payloadmass) as AveragePayloadMass, 
        count(boosterversion) as LaunchTotal
from spacexlaunch
group by boosterversion
order by AveragePayloadMass desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,MinPayloadMass,MaxPayloadMass,TotalPayloadMass,AveragePayloadMass,LaunchTotal
F9 B5,325.0,17400.0,1615075.3265306123,10625.495569280343,152
F9 v1.0,525.0,8933.290816326531,27968.581632653062,5593.716326530613,5
F9 B4,362.0,9600.0,63575.29081632653,5297.940901360545,12
F9 FT,475.0,9600.0,113485.29081632654,4728.553784013606,24
F9 v1.1,500.0,4707.0,38020.0,2534.6666666666665,15


In [101]:
%%sql
select boosterversion, landingstatus, count(boosterversion) as TotalFlights
from spacexlaunch
group by boosterversion, landingstatus

 * sqlite:///SpaceX.db
Done.


BoosterVersion,Landingstatus,TotalFlights
F9 B4,0,6
F9 B4,1,6
F9 B5,0,9
F9 B5,1,143
F9 FT,0,6
F9 FT,1,18
F9 v1.0,0,5
F9 v1.1,0,12
F9 v1.1,1,3


4. How many type of booster landing?

* A controlled ocean landing means the ocean test controlled descent, for the sole purpose of gathering test data, such boosters were destroyed at sea. Controlled means success test, soft vertical ocean touchdown. Uncontrolled means failure test, ocean touchdown control failed.

* Drone ship: land on a drone ship at sea

* Ground pad: land on a ground pad near the launch site

* Failure parachute: attempt to recover the first stage by parachuting it into the ocean but fail

* The first success date of ground pad is 2015-12-22, dron ship is 2016-04-08, and ocean is 2014-04-18

* Success droneship & groundpad is B5, B4, FT, success ocean focus on FT, v1.1

In [103]:
%%sql
select boosterlanding, count(boosterlanding) as LaunchTotal
from spacexlaunch
group by boosterlanding

 * sqlite:///SpaceX.db
Done.


Boosterlanding,LaunchTotal
Controlled(ocean),5
Failure(drone ship),8
Failure(ground pad),1
Failure(parachute),2
No attempt,24
Precluded(drone ship),1
Success(drone ship),136
Success(ground pad),29
Uncontrolled(ocean),2


In [102]:
%%sql
select boosterlanding, min("Dateandtime(UTC)") as FirstSuccessDate
from spacexlaunch
where boosterlanding = "Success(ground pad)"

 * sqlite:///SpaceX.db
Done.


Boosterlanding,FirstSuccessDate
Success(ground pad),2015-12-22


In [104]:
%%sql
select boosterlanding, min("Dateandtime(UTC)") as FirstSuccessDate
from spacexlaunch
where boosterlanding = "Success(drone ship)"

 * sqlite:///SpaceX.db
Done.


Boosterlanding,FirstSuccessDate
Success(drone ship),2016-04-08


In [105]:
%%sql
select boosterlanding, min("Dateandtime(UTC)") as FirstSuccessDate
from spacexlaunch
where boosterlanding = "Controlled(ocean)"

 * sqlite:///SpaceX.db
Done.


Boosterlanding,FirstSuccessDate
Controlled(ocean),2014-04-18


In [108]:
%%sql
select boosterversion, count(boosterversion) as TotalSuccess
from spacexlaunch
where boosterlanding = "Success(ground pad)"
group by boosterversion
order by TotalSuccess desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,TotalSuccess
F9 B5,20
F9 FT,6
F9 B4,3


In [109]:
%%sql
select boosterversion, count(boosterversion) as TotalSuccess
from spacexlaunch
where boosterlanding = "Success(drone ship)"
group by boosterversion
order by TotalSuccess desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,TotalSuccess
F9 B5,123
F9 FT,10
F9 B4,3


In [110]:
%%sql
select boosterversion, count(boosterversion) as TotalSuccess
from spacexlaunch
where boosterlanding = "Controlled(ocean)"
group by boosterversion
order by TotalSuccess desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,TotalSuccess
F9 v1.1,3
F9 FT,2


5. 2022 launches

* In 2022, SpaceX landing site focuses CCSFS accounted for 50% flights (30/60), KSC has 17, VSFB has 13 flights. SpaceX maintain each month launch from 4 to 6 flights, December reaches the highest number of launches in year 7 flights in a month

* Falcon 9 Block 5 first stage landing success almost the time, there is 2 no attempt flights in November but is recorded as success on wiki

* Last year SpaceX focuses on LEO, SSO, GTO

In [119]:
%%sql 
select substr("Dateandtime(UTC)", 6, 2) as month, launchsite, count(launchsite) as TotalFlights
from spacexlaunch
where substr("Dateandtime(UTC)", 1, 4) = '2022'
group by month, launchsite

 * sqlite:///SpaceX.db
Done.


month,Launchsite,TotalFlights
01,CCSFS SLC-40,2
01,KSC LC-39A,2
02,CCSFS SLC-40,1
02,KSC LC-39A,1
02,VSFB SLC-4E,2
03,CCSFS SLC-40,2
03,KSC LC-39A,1
04,CCSFS SLC-40,3
04,KSC LC-39A,2
04,VSFB SLC-4E,1


In [120]:
%%sql 
select substr("Dateandtime(UTC)", 6, 2) as month, landingstatus, count(landingstatus) as TotalFlights
from spacexlaunch
where substr("Dateandtime(UTC)", 1, 4) = '2022'
group by month, landingstatus

 * sqlite:///SpaceX.db
Done.


month,Landingstatus,TotalFlights
01,1,4
02,1,4
03,1,3
04,1,6
05,1,5
06,1,5
07,1,6
08,1,6
09,1,4
10,1,6


In [126]:
%%sql 
select orbit, count(orbit) as TotalFlights, avg(payloadmass) as AvgPayloadMass
from spacexlaunch
where substr("Dateandtime(UTC)", 1, 4) = '2022'
group by orbit
order by TotalFlights desc
limit 3

 * sqlite:///SpaceX.db
Done.


Orbit,TotalFlights,AvgPayloadMass
LEO,39,13548.13566195709
SSO,10,9843.816326530612
GTO,7,5150.142857142857
